In [ ]:
%%capture
!pip install asteroid

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/TG-Separación-Fuentes/code/train-speech-separation-models/train

/content/drive/Shareddrives/TG-Separación-Fuentes/code/train-speech-separation-models/train


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os
import torch
import random as random
from torch.utils.data import DataLoader,Dataset
import pytorch_lightning as pl
import yaml
import json
from asteroid import ConvTasNet
from asteroid.utils import prepare_parser_from_dict, parse_args_as_dict
import argparse
from asteroid.metrics import get_metrics
from asteroid.utils import tensors_to_device
from tqdm import tqdm
from asteroid.dsp.normalization import normalize_estimates
import soundfile as sf
from IPython.display import display, Audio
from asteroid.models import BaseModel
from asteroid.losses import PITLossWrapper, pairwise_neg_sisdr
from lib.CallSpanish_dataset import CallSpanish

In [ ]:
# CSV
ROOT_CSV  ="/content/drive/Shareddrives/TG-Separación-Fuentes/code/"
PATH_CSV_MIX = ROOT_CSV +"mixture_callfriend_spanish.csv"
PATH_CSV_TRAIN = ROOT_CSV + "mixture_train_mix_clean_callfriend_spanish.csv"
PATH_CSV_VALID = ROOT_CSV +"mixture_val_mix_clean_callfriend_spanish.csv"
PATH_CSV_TEST = ROOT_CSV + "mixture_test_mix_clean_callfriend_spanish.csv"
PATH_CONFIG = "./resources/conf.yml"

df_val = pd.read_csv(PATH_CSV_VALID)

In [ ]:
df_val.shape

(87441, 5)

In [ ]:
with open(PATH_CONFIG) as f:
        conf = yaml.safe_load(f)

In [ ]:
PATH_TEST = df_val.iloc[1,1]  
model_before = BaseModel.from_pretrained("JorisCos/ConvTasNet_Libri2Mix_sepclean_8k")
model_before.cuda()

In [ ]:
mixture, _ = sf.read(PATH_TEST, dtype="float32", always_2d=True)
# Soundfile returns the mixture as shape (time, channels), and Asteroid expects (batch, channels, time)
mixture = mixture.transpose()
mixture = mixture.reshape(1, mixture.shape[0], mixture.shape[1])
out_wavs_before = model_before.separate(mixture)

In [ ]:
display(Audio(PATH_TEST))
display(Audio(out_wavs_before[0,0,:],rate=8000))
display(Audio(out_wavs_before[0,1,:],rate=8000))

In [ ]:
model = ConvTasNet.from_pretrained("JorisCos/ConvTasNet_Libri2Mix_sepclean_8k")
model.cuda()

In [ ]:
loss_func = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")

In [ ]:
path_s1 = df_val.loc[1,"source_1_path"]
path_s2 = df_val.loc[1,"source_2_path"]
path_s1,path_s2

('/content/drive/Shareddrives/TG-Separación-Fuentes/Datasets/CallFriend-Spanish-Prepro/Train/source_1/4019_33.wav',
 '/content/drive/Shareddrives/TG-Separación-Fuentes/Datasets/CallFriend-Spanish-Prepro/Train/source_2/4019_33.wav')

In [ ]:
s1, _ = sf.read(path_s1, dtype="float32")
s2,_ =  sf.read(path_s2, dtype="float32")

In [ ]:
start = 4879
end = 4880

In [ ]:
test_set = CallSpanish(
    csv_path=PATH_CSV_VALID,
    task="sep_clean",
    sample_rate=8000,
    n_src=2,
    segment=None,
    return_id=True,
    start = start,
    end = end
)

In [ ]:
model = model_before

In [ ]:
model_device = next(model.parameters()).device
loss_func = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")
COMPUTE_METRICS = ["si_sdr", "sdr", "sir", "sar", "stoi"]

In [ ]:
torch.no_grad().__enter__()
for idx in tqdm(range(len(test_set))):
    # Forward the network on the mixture.
    mix, sources, ids = test_set[idx]
    mix, sources = tensors_to_device([mix, sources], device=model_device)
    est_sources = model(mix.unsqueeze(0))
    loss, reordered_sources = loss_func(est_sources, sources[None], return_est=True)
    mix_np = mix.cpu().data.numpy()
    sources_np = sources.cpu().data.numpy()
    est_sources_np = reordered_sources.squeeze(0).cpu().data.numpy()

    utt_metrics_1 = get_metrics(
                mix_np,
                sources_np,
                est_sources_np,
                sample_rate=conf["data"]["sample_rate"],
                metrics_list=COMPUTE_METRICS,
            )

100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


In [ ]:
PATH_TESTT = df_val.iloc[4879,1]
display(Audio(PATH_TESTT))
display(Audio(est_sources_np[0],rate=8000))
display(Audio(est_sources_np[1],rate=8000))

In [ ]:
utt_metrics_1

{'input_sar': 278.82050243783317,
 'input_sdr': -0.028528897432106515,
 'input_si_sdr': -0.0587313175201416,
 'input_sir': -0.028528897432106515,
 'input_stoi': 0.8225752264244054,
 'sar': 12.078309445897384,
 'sdr': 10.493667489878783,
 'si_sdr': 9.976849555969238,
 'sir': 16.796804215964507,
 'stoi': 0.8737822242905939}

In [ ]:
utt_metrics_1

{'input_sar': 278.82050243783317,
 'input_sdr': -0.028528897432106515,
 'input_si_sdr': -0.0587313175201416,
 'input_sir': -0.028528897432106515,
 'input_stoi': 0.8225752264244054,
 'sar': 12.078309445897384,
 'sdr': 10.493667489878783,
 'si_sdr': 9.976849555969238,
 'sir': 16.796804215964507,
 'stoi': 0.8737822242905939}

In [ ]:
mix, sources = tensors_to_device([test_set[0][0],test_set[0][1]], device=model_device)
est_sources = model(mix[None, None])
loss, reordered_sources = loss_func(est_sources, sources[None], return_est=True)
mix_np = mix[None].cpu().data.numpy()
sources_np = sources.cpu().data.numpy()
est_sources_np = reordered_sources.squeeze(0).cpu().data.numpy()
utt_metrics_2 = get_metrics(
    mix_np,
    sources_np,
    est_sources_np,
    sample_rate=conf["data"]["sample_rate"],
    metrics_list=COMPUTE_METRICS,
)

In [ ]:
utt_metrics_2

{'input_sar': 278.82050243783317,
 'input_sdr': -0.028528897432106515,
 'input_si_sdr': -0.0587313175201416,
 'input_sir': -0.028528897432106515,
 'input_stoi': 0.8225752264244054,
 'sar': 12.078309445897384,
 'sdr': 10.493667489878783,
 'si_sdr': 9.976849555969238,
 'sir': 16.796804215964507,
 'stoi': 0.8737822242905939}

In [ ]:
PATH_CONFIG_DPRNN = "./resources/conf-DPRNN.yml"

In [ ]:
with open(PATH_CONFIG_DPRNN) as f:
        conf = yaml.safe_load(f)

In [ ]:
conf

{'data': {'mode': 'min',
  'nondefault_nsrc': None,
  'sample_rate': 8000,
  'segment': 2.0,
  'task': 'sep_clean',
  'train_dir': 'data/wav8k/min/tr/',
  'valid_dir': 'data/wav8k/min/cv/'},
 'filterbank': {'kernel_size': 2, 'n_filters': 64, 'stride': 1},
 'masknet': {'bidirectional': True,
  'bn_chan': 128,
  'chunk_size': 250,
  'dropout': 0,
  'hid_size': 128,
  'hop_size': 125,
  'in_chan': 64,
  'mask_act': 'sigmoid',
  'n_repeats': 6,
  'n_src': 2,
  'out_chan': 64},
 'optim': {'lr': 0.001, 'optimizer': 'adam', 'weight_decay': 1e-05},
 'test': {'n_save_examples': 5},
 'training': {'batch_size': 4,
  'early_stop': True,
  'epochs': 200,
  'gradient_clipping': 5,
  'half_lr': True,
  'num_workers': 4}}

In [ ]:
PATH_TEST = df_val.iloc[1,1]
model_DPRNN = BaseModel.from_pretrained("mpariente/DPRNNTasNet-ks2_WHAM_sepclean")
model_DPRNN.cuda()

In [ ]:
model_device = next(model_DPRNN.parameters()).device

In [ ]:
torch.no_grad().__enter__()
for idx in tqdm(range(len(test_set))):
    # Forward the network on the mixture.
    mix, sources, ids = test_set[idx]
    mix, sources = tensors_to_device([mix,sources], device=model_device)
    est_sources = model_DPRNN(mix[None, None])
    loss, reordered_sources = loss_func(est_sources, sources[None], return_est=True)
    mix_np = mix[None].cpu().data.numpy()
    sources_np = sources.cpu().data.numpy()
    est_sources_np = reordered_sources.squeeze(0).cpu().data.numpy()
    utt_metrics_DPRNN = get_metrics(
        mix_np,
        sources_np,
        est_sources_np,
        sample_rate=conf["data"]["sample_rate"],
        metrics_list=COMPUTE_METRICS,
    )

100%|██████████| 1/1 [00:02<00:00,  2.22s/it]


In [ ]:
utt_metrics_DPRNN

{'input_sar': 279.8306388490538,
 'input_sdr': 0.20778359953294895,
 'input_si_sdr': 0.0657612681388855,
 'input_sir': 0.20778359953295159,
 'input_stoi': 0.8274112745763731,
 'sar': 5.023151780034536,
 'sdr': 0.6795118673945548,
 'si_sdr': -0.42777013778686523,
 'sir': 4.013898936333515,
 'stoi': 0.6978179947389691}

In [ ]:
est_sources_np.shape

(2, 120000)

In [ ]:
PATH_TESTT = df_val.iloc[4879,1]
display(Audio(PATH_TESTT))
display(Audio(est_sources_np[0],rate=8000))
display(Audio(est_sources_np[1],rate=8000))

In [ ]:
display(Audio(est_sources_np[0],rate=8000))
display(Audio(est_sources_np[1],rate=8000))

In [ ]:
utt_metrics_2

{'input_sar': 278.82050243783317,
 'input_sdr': -0.028528897432106515,
 'input_si_sdr': -0.0587313175201416,
 'input_sir': -0.028528897432106515,
 'input_stoi': 0.8225752264244054,
 'sar': 12.078309445897384,
 'sdr': 10.493667489878783,
 'si_sdr': 9.976849555969238,
 'sir': 16.796804215964507,
 'stoi': 0.8737822242905939}

In [ ]:
torch.no_grad().__enter__()
for idx in tqdm(range(len(test_set))):
    # Forward the network on the mixture.
    mix, sources, ids = test_set[idx]
    mix, sources = tensors_to_device([mix, sources], device=model_device)
    est_sources = model_DPRNN(mix.unsqueeze(0))
    loss, reordered_sources = loss_func(est_sources, sources[None], return_est=True)
    mix_np = mix.cpu().data.numpy()
    sources_np = sources.cpu().data.numpy()
    est_sources_np = reordered_sources.squeeze(0).cpu().data.numpy()

    utt_metrics_2_DPRNN = get_metrics(
                mix_np,
                sources_np,
                est_sources_np,
                sample_rate=conf["data"]["sample_rate"],
                metrics_list=COMPUTE_METRICS,
            )

100%|██████████| 1/1 [00:02<00:00,  2.23s/it]
